<a href="https://colab.research.google.com/github/athammad/ateTrees/blob/main/CTL_ATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install causal_tree_learn

     |████████████████████████████████| 122kB 2.6MB/s 
  Created wheel for causal-tree-learn: filename=causal_tree_learn-2.41-cp36-cp36m-linux_x86_64.whl size=406195 sha256=9b8ab15125192e8afaa83a9790c87c4519685c8ae696f37d35c6049b1708c90c
  Stored in directory: /root/.cache/pip/wheels/4d/c1/c5/c035132edf632cef6fe046cbab48b5a9e2ccc46bbd0cba1b1c
Successfully built causal-tree-learn


In [ ]:
import numpy as np

def bootstrap(data, n=1000, func=np.mean):
    """
    Generate `n` bootstrap samples, evaluating `func`
    at each resampling. `bootstrap` returns a function,
    which can be called to obtain confidence intervals
    of interest.
    """
    simulations = list()
    sample_size = len(data)
    xbar_init = np.mean(data)
    for c in range(n):
        itersample = np.random.choice(data, size=sample_size, replace=True)
        simulations.append(func(itersample))
    simulations.sort()
    def ci(p):
        """
        Return 2-sided symmetric confidence interval specified
        by p.
        """
        u_pval = (1+p)/2.
        l_pval = (1-u_pval)
        l_indx = int(np.floor(n*l_pval))
        u_indx = int(np.floor(n*u_pval))
        return(simulations[l_indx],simulations[u_indx])
    return(ci)

In [ ]:


import pandas as pd
from CTL.causal_tree_learn import CausalTree
from sklearn.model_selection import train_test_split
import numpy as np
import statistics 

asthma = pd.read_csv('https://raw.githubusercontent.com/edgeslab/CTL/master/data/asthma.txt', delimiter=' ', index_col=None)

asthma.columns = ['physician', 'age', 'sex', 'education', 'insurance', 'drug coverage', 'severity',
                  'comorbidity', 'physical comorbidity', 'mental comorbidity', 'satisfaction']


y = asthma['satisfaction'].values
treatment = asthma['physician'].values

x = asthma.drop(['satisfaction', 'physician'], axis=1).values

columns = asthma.drop(['satisfaction', 'physician'], axis=1).columns

y[y == 0] = -1

treatment[treatment == 1] = 0
treatment[treatment == 2] = 1


#np.random.seed(0)
resultCTL= []

for i in range(100):
  x_train, x_test, y_train, y_test, treat_train, treat_test = train_test_split(x, y, treatment,
                                                                             test_size=0.5,random_state=i)
  
  # val honest CTL
  cthv = CausalTree(val_honest=True)
  cthv.fit(x_train, y_train, treat_train)
  cthv.prune()
  cthv_predict = cthv.predict(x_test)

  
  resultMean=statistics.mean(cthv_predict)
  resultCTL.append(resultMean)



boot = bootstrap(resultCTL, n=5000)
cintervals = [boot(i) for i in (.90, .95, .99, .995)]

import statistics 
print(statistics.mean(resultCTL))
print(cintervals)




-0.3337509646763404
[(-0.36199089530477496, -0.30442462146930077), (-0.36693727540149207, -0.29896505999932255), (-0.37752821700944417, -0.289378810994651), (-0.3828225359676973, -0.2840006528557211)]
